In [1]:
setwd("/staging/leuven/stg_00041/Bradley/")

library(umap)
library(ggplot2)
library(Seurat)
library(reshape2)
library(plyr)
library(dplyr)


Attaching SeuratObject


Attaching package: 'dplyr'


The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
load("combinedMeta.Robject")

In [3]:
load("integratedBlastoids.combinedCounts.raw.Robject")

In [4]:
manualCalculatePercentMT <- function(cellGeneCounts, geneNames){
    geneNames <- as.character(geneNames)
    return((sum(cellGeneCounts[grep(pattern="^MT-", x = geneNames)])/sum(cellGeneCounts))*100)
    
    
}

In [8]:
for(i in 1:4){
    combinedCounts[[i]]$RNAcounts <-apply(combinedCounts[[i]]$counts[,which(colnames(combinedCounts[[i]]$counts)!="gene")], 2, sum)
}

In [ ]:
for(i in 1:4){
    combinedCounts[[i]]$percentMT <- apply(combinedCounts[[i]]$counts[,which(colnames(combinedCounts[[i]]$counts)!="gene")], 2, manualCalculatePercentMT,
                                          geneNames=rownames(combinedCounts[[i]]$counts))
}

In [ ]:
#filtering for MT % and read count
applyFilters <- function(combinedCountsPiece){
    filtered <-list()
    
    survivesFilter <- which(combinedCountsPiece$RNAcounts> 
                                                         10^(log10(summary(combinedCountsPiece$RNAcounts)[2])-IQR(log10(combinedCountsPiece$RNAcounts)) ) &
                                                        combinedCountsPiece$percentMT<10)
    
    filtered$counts <- combinedCountsPiece$counts[,survivesFilter]
    filtered$annotation <- combinedCountsPiece$annotation[survivesFilter,]
    filtered$RNAcounts <- combinedCountsPiece$RNAcounts[survivesFilter]
    filtered$percentMT <- combinedCountsPiece$percentMT[survivesFilter]
    
    
    filtered$counts <- as.data.frame(filtered$counts)
    filtered$counts$gene <- row.names(filtered$counts)
    return(filtered)
}





#below is my filter for RNAcounts, it is quartile1-1.5*the interquartile distance, but in log10 because only log transformed do my graphs have a more normal distribution
#10^(log10(summary(combinedCounts[[i]]$RNAcounts)[2])-IQR(log10(combinedCounts[[i]]$RNAcounts)) )
#this might be horribl statistics but it looks decent to me.


In [ ]:
allMerge <- function(studies){
    combinedCountsMerge <- merge(applyFilters(combinedCounts[[studies[1]]])$counts,
                                applyFilters(combinedCounts[[studies[2]]])$counts, by="gene", all=F)
    if(length(studies)>2){
        for(i in 3:length(studies)){
            combinedCountsMerge <- merge(combinedCountsMerge, applyFilters(combinedCounts[[studies[i]]])$counts, by="gene", all=F)
        }
    }
    return(combinedCountsMerge)
}

In [ ]:
#I had this as a function when I had more datasets and was trying different combinations
integrateAsFunction <- function(studyNums){

combinedCountsMerge <- allMerge(studyNums) 

rownames(combinedCountsMerge) <- combinedCountsMerge$gene

combinedCountsMerge <- combinedCountsMerge[apply(combinedCountsMerge[,which(colnames(combinedCountsMerge)!="gene")], 1, function(x){!all(x==0)}),]

    
    percentMT <- c()
    
    for(i in studyNums){
       percentMT <- c(percentMT, combinedCounts[[i]]$percentMT)
                      
    }
    
    percentMT <- data.frame(percentMT, cell=names(percentMT))
    
combinedCountsSeurat<- CreateSeuratObject(
  counts = combinedCountsMerge[,which(colnames(combinedCountsMerge)!="gene")],
  project = "combinedCountsSeurat", 
  min.cells = 0,
  min.features = 0
  )



combinedCountsSeurat@meta.data$cell <- rownames(combinedCountsSeurat@meta.data)

combinedCountsSeurat@meta.data <- join(combinedCountsSeurat@meta.data, combinedMeta[,], by="cell", type="left")

combinedCountsSeurat@meta.data <- join(combinedCountsSeurat@meta.data, percentMT, by="cell", type="left")

    
rownames(combinedCountsSeurat@meta.data) <- combinedCountsSeurat@meta.data$cell

combinedCountsSeurat <- NormalizeData(object=combinedCountsSeurat, normalization.method="LogNormalize", scale.factor=10000)
combinedCountsSeurat <- ScaleData(combinedCountsSeurat, verbose=FALSE)
combinedCountsSeurat <- FindVariableFeatures(combinedCountsSeurat, selection.method="vst", nfeatures=2000)
combinedCountsSeurat <- RunPCA(combinedCountsSeurat, npcs=30, verbose=FALSE)
combinedCountsSeurat <- RunUMAP(combinedCountsSeurat, reduction="pca", dims=1:30)
combinedCountsSeurat <- FindNeighbors(combinedCountsSeurat, reduction="pca", dims=1:30)

combinedCountsSeuratSplit <- SplitObject(combinedCountsSeurat, split.by="study")

combinedCountsSeuratSplit <- lapply(combinedCountsSeuratSplit, function(x){
    x <- NormalizeData(x)
    x <- FindVariableFeatures(x, selection.method="vst", nfeatures = 2000)
})

features <- SelectIntegrationFeatures(object.list=combinedCountsSeuratSplit)

anchors <- FindIntegrationAnchors(combinedCountsSeuratSplit, anchor.features=features)

combinedCountsSeuratReintegrated <- IntegrateData(anchorset=anchors, dims=1:50)

combinedCountsSeuratReintegrated <- ScaleData(combinedCountsSeuratReintegrated, verbose=FALSE)
combinedCountsSeuratReintegrated <- RunPCA(combinedCountsSeuratReintegrated, npcs=30, verbose=FALSE)

combinedCountsSeuratReintegrated <- FindNeighbors(combinedCountsSeuratReintegrated, reduction="pca", dims=1:30)

combinedCountsSeuratReintegrated <- suppressMessages(RunUMAP(combinedCountsSeuratReintegrated, reduction="pca", dims=1:30, seed.use = 123))

    return(combinedCountsSeuratReintegrated)
}

In [ ]:
allDataIntegration.seurat <- integrateAsFunction(c(1:4))
save(allDataIntegration.seurat, file="allDatasets.combinedCountsSeuratReintegrated.filtered.Robject")

In [ ]:
tyserPetRivron.seurat <- integrateAsFunction(c(1:3))
save(tyserPetRivron.seurat, file="tyserPetRivron.combinedCountsSeuratReintegrated.filtered.Robject")
